## Import Tools

In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

## Dataset Processing 

### Read in the original dataset 

In [2]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

In [3]:
print(train_label)

tensor([1, 1, 0,  ..., 0, 0, 1])


In [4]:
print(train_label.shape)

torch.Size([15000])


### CSI Processing: Take Modulus of complex matrices

In [5]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [6]:
print(train_CSI_modulus)

tensor([[[[100.6578, 124.7878, 106.1179,  ..., 304.7704, 299.6064, 324.3594],
          [132.8157, 106.6771,  91.2688,  ..., 269.1561, 323.5568, 299.9617],
          [129.1395, 148.4756, 170.0735,  ..., 399.8112, 407.4420, 402.0112],
          [ 74.0000,  71.4493,  59.3633,  ..., 134.0149, 129.6919, 124.0363]]],


        [[[177.0198, 170.4963, 169.1065,  ...,  46.6154,  37.6431,  64.4981],
          [143.6802, 143.0874,  88.0909,  ...,  44.0454,  22.2036,  27.6586],
          [ 97.8008,  80.7527,  71.7008,  ...,  32.2025,  22.4722,  39.3573],
          [ 39.8121,  45.7930,  31.6228,  ...,  16.5529,   8.0623,  25.6125]]],


        [[[411.3940, 421.5412, 380.1276,  ..., 509.8431, 550.0582, 539.8120],
          [366.8079, 387.3629, 353.0340,  ..., 596.1241, 619.6975, 605.5353],
          [574.8991, 593.8560, 612.0008,  ..., 928.9521, 923.3618, 914.0552],
          [289.8362, 287.2368, 281.0427,  ..., 354.9113, 339.0634, 333.9461]]],


        ...,


        [[[296.5889, 288.2672, 292.76

In [7]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


###  CSI Processing: Normalize to [0,1]

In [8]:
# Min-Max Scaling
min_value = torch.min(train_CSI_modulus)
max_value = torch.max(train_CSI_modulus)

normalized_train_CSI_modulus = (train_CSI_modulus - min_value) / (max_value - min_value)
normalized_valid_CSI_modulus = (valid_CSI_modulus - min_value) / (max_value - min_value)


### ML Classifcation w/ KNN

In [9]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [10]:
# Convert 4d to 2d
train_data_2d = normalized_train_CSI_modulus.view(normalized_train_CSI_modulus.size(0), -1)  # Reshape to (15000, 4 * 1632)
valid_data_2d = normalized_valid_CSI_modulus.view(normalized_valid_CSI_modulus.size(0), -1)  # Reshape to (15000, 4 * 1632)

In [11]:
# Shuffle and split 
x, y = np.array(train_data_2d), np.array(train_label)
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42) 


In [12]:
sq = int(np.sqrt(15000))
knn = KNeighborsClassifier(n_neighbors=35)
n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

train_scores = []
valid_scores = []
for train_index, valid_index in kf.split(X_train, y_train):
    X_train_fold, X_valid_fold = X_train[train_index], X_train[valid_index]
    y_train_fold, y_valid_fold = y_train[train_index], y_train[valid_index]

    knn.fit(X_train_fold, y_train_fold)

    train_score = knn.score(X_train_fold, y_train_fold)
    valid_score = knn.score(X_valid_fold, y_valid_fold)

    train_scores.append(train_score)
    valid_scores.append(valid_score)

mean_train_score = np.mean(train_scores)
mean_valid_score = np.mean(valid_scores)

print('Mean accuracy of KNN classifier on training set: {:.2f}'.format(mean_train_score))
print('Mean accuracy of KNN classifier on validation set: {:.2f}'.format(mean_valid_score))


C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavio

Mean accuracy of KNN classifier on training set: 0.96
Mean accuracy of KNN classifier on validation set: 0.94


C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### ANN approach (MLP)

In [13]:
my_device = None
if torch.cuda.is_available():
    my_device = torch.device("cuda")
elif torch.backends.mps.is_available():
    torch.backends.mps.is_built()
    my_device = torch.device("mps")
else:
    my_device = torch.device("cpu")
print("Running on: ",my_device)

Running on:  cpu


- Instantiate a Neural Network Model

In [36]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(4 * 1632, 2048)
        self.bn1 = nn.BatchNorm1d(2048)
        
        self.layer2 = nn.Linear(2048, 2048)
        self.bn2 = nn.BatchNorm1d(2048)
        
        self.layer3 = nn.Linear(2048, 1024)
        self.bn3 = nn.BatchNorm1d(1024)
        
        self.layer4 = nn.Linear(1024, 512)
        self.bn4 = nn.BatchNorm1d(512)
        
        self.layer5 = nn.Linear(512, 256)
        self.bn5 = nn.BatchNorm1d(256)
        
        self.layer6 = nn.Linear(256, 64)
        self.bn6 = nn.BatchNorm1d(64)
        
        self.layer7 = nn.Linear(64, 32)
        self.bn7 = nn.BatchNorm1d(32)
        
        self.layer8 = nn.Linear(32, 16)
        self.bn8 = nn.BatchNorm1d(16)
        
        self.layer9 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.layer1(x)))
        x = F.relu(self.bn2(self.layer2(x)))
        x = F.relu(self.bn3(self.layer3(x)))
        x = F.relu(self.bn4(self.layer4(x)))
        x = F.relu(self.bn5(self.layer5(x)))
        x = F.relu(self.bn6(self.layer6(x)))
        x = F.relu(self.bn7(self.layer7(x)))
        x = F.relu(self.bn8(self.layer8(x)))
        x = self.layer9(x)
        return x



- Add a loss function and an optimizer

- Train the neural network

In [40]:
import torch.optim as optim
import copy
from sklearn.model_selection import KFold  
import torch.nn.init as init
from torch.optim.lr_scheduler import StepLR



num_folds = 2
kf = KFold(n_splits=num_folds)
best_model_states = []
best_losses = []
torch.autograd.set_detect_anomaly(True)

for train_index, valid_index in kf.split(train_data_2d):
    # Split the data into training and validation sets for each fold
    X_train_fold, X_valid_fold = train_data_2d[train_index], train_data_2d[valid_index]
    y_train_fold, y_valid_fold = train_label[train_index], train_label[valid_index]

    net = Net() 
     
    for m in net.modules():
        if isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    optimizer = optim.SGD(net.parameters(), lr=0.08)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    criterion = nn.BCEWithLogitsLoss()

    best_loss = float('inf')  
    best_model_state_dict = None

    num_epochs = 50

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = net(X_train_fold)
        loss = criterion(output, y_train_fold)

        loss.backward()
        optimizer.step()
        scheduler.step()
        print('Epoch %d/%d, Training Loss: %.4f' % (epoch+1, num_epochs, loss.item()))
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model_state_dict = copy.deepcopy(net.state_dict())

    best_model_states.append(best_model_state_dict)
    best_losses.append(best_loss)

best_fold = best_losses.index(min(best_losses))
best_model_state_dict = best_model_states[best_fold]
net = Net()  # Initialize a new model
net.load_state_dict(best_model_state_dict)


Epoch 1/50, Training Loss: 1.8078
Epoch 2/50, Training Loss: 1.0102
Epoch 3/50, Training Loss: 0.5564
Epoch 4/50, Training Loss: 0.4867
Epoch 5/50, Training Loss: 0.4334
Epoch 6/50, Training Loss: 0.3068
Epoch 7/50, Training Loss: 0.2564
Epoch 8/50, Training Loss: 0.2255
Epoch 9/50, Training Loss: 0.2056
Epoch 10/50, Training Loss: 0.1915
Epoch 11/50, Training Loss: 0.1830
Epoch 12/50, Training Loss: 0.1716
Epoch 13/50, Training Loss: 0.1662
Epoch 14/50, Training Loss: 0.1615
Epoch 15/50, Training Loss: 0.1571
Epoch 16/50, Training Loss: 0.1531
Epoch 17/50, Training Loss: 0.1511
Epoch 18/50, Training Loss: 0.1493
Epoch 19/50, Training Loss: 0.1475
Epoch 20/50, Training Loss: 0.1457
Epoch 21/50, Training Loss: 0.1440
Epoch 22/50, Training Loss: 0.1431
Epoch 23/50, Training Loss: 0.1423
Epoch 24/50, Training Loss: 0.1415
Epoch 25/50, Training Loss: 0.1407
Epoch 26/50, Training Loss: 0.1399
Epoch 27/50, Training Loss: 0.1395
Epoch 28/50, Training Loss: 0.1391
Epoch 29/50, Training Loss: 0

<All keys matched successfully>

In [41]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

net.eval()
with torch.no_grad():
    train_predictions = net(train_data_2d)
    test_predictions = net(valid_data_2d)

# Apply sigmoid activation
train_predictions = torch.sigmoid(train_predictions)
test_predictions = torch.sigmoid(test_predictions)

# Round to 0 or 1
train_predictions = torch.round(train_predictions)
test_predictions = torch.round(test_predictions)

print("Training Accuracy:", accuracy_score(train_label, train_predictions.numpy()))
print("Validation Accuracy:", accuracy_score(valid_label, test_predictions.numpy()))
print("Training R-squared:", r2_score(train_label, train_predictions))
print("Validation R-squared:", r2_score(valid_label, test_predictions))
print("Training MSE:", mean_squared_error(train_label, train_predictions))
print("Validation MSE:", mean_squared_error(valid_label, test_predictions))
print("Training MAE:", mean_absolute_error(train_label, train_predictions))


Training Accuracy: 0.9775333333333334
Validation Accuracy: 0.9732
Training R-squared: 0.8459987017145988
Validation R-squared: 0.823865165892725
Training MSE: 0.022466667
Validation MSE: 0.0268
Training MAE: 0.022466667
